In [1]:
import re
import nltk
import pandas as pd
import numpy as np
from pprint import pprint


# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, TfidfModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)


In [3]:
sw = open("../src/stopwords.txt", "r")
stopwords = sw.read()
stopwords_list = stopwords.split(", ")
stopwords_set = set(stopwords_list)
sw.close()

In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(stopwords_list)


In [5]:
regular_episodes = pd.read_csv("../data/jeopardy_regular_episodes.csv")
regular_episodes = regular_episodes.sample(frac=.05)
type(regular_episodes['Question and Answer'])

pandas.core.series.Series

## Cleaning

In [6]:
def convert_col_to_list(df, col='Question and Answer'):
    """
    takes in a column from a dataframe and
    returns a list of text
    Args:
        df (Pandas DataFrame):
        col (str): column name. Default to "Question and Answer"
    Returns:
        a list of strings, where each column entry is one string
        to be tokenized later
    """
    text = df[col].values.tolist()
    return text

In [7]:
def remove_punc(texts):
    '''
    texts are a list of a string where each string is a row of text from the df
    '''
    clues = [re.sub('\S*@\S*\s?', '', sent) for sent in texts]

    # Remove new line characters
    clues = [re.sub('\s+', ' ', sent) for sent in clues]

    # Remove distracting single quotes
    clues = [re.sub("\'", "", sent) for sent in clues]
    return clues
    

In [8]:
def remove_punc2(texts):
    x = [''.join(c for c in s if c not in string.punctuation) for s in texts]
    x = [s for s in x if s]
    return x

In [9]:
#Generator function for tokenizing and removing punctuation
# deacc=True removes punctuation

def tokenize(clues):
    for clue in clues:
        yield(gensim.utils.simple_preprocess(str(clue), deacc=True))
    

In [10]:
def make_bigrams(texts):
    clue_words = list(tokenize(texts))
    bigram = gensim.models.Phrases(clue_words, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

In [11]:
def make_trigrams(texts):
    clue_words = list(tokenize(texts))
    trigram = gensim.models.Phrases(bigram[clue_words], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

In [12]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts, stop_words):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]


def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [13]:
def lemmatization_1(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """
    """
    nlp = spacy.load('en', disable=['parser', 'ner'])
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [14]:
text = convert_col_to_list(regular_episodes, col='Question and Answer')

clues = remove_punc(text)
clue_words = list(tokenize(clues))

# Remove Stop Words and returns a list of tokens as long as clue_words are tokenized 

nostops = remove_stopwords(clue_words, stop_words)

# Form Bigrams
clue_words_bigrams = make_bigrams(nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
# clue_lemmatized = lemmatization(clue_words_bigrams,allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Lemmatize without the allowed_postages
clue_lemmatized = lemmatization(clue_words_bigrams)
clue_lemmatized = remove_stopwords(clue_lemmatized, stop_words)

print(clue_lemmatized[:10])

[['tree'], ['let', 'response'], ['defame', 'print', 'libel', 'verbally'], ['linen', 'old', 'textile', 'fabric', 'stock', 'flax', 'plant'], ['story', 'oneact', 'drama'], ['hire', 'little', 'clean', 'smoke', 'filling', 'room', 'still'], ['partly', 'enclose'], ['famous', 'inventor', 'molecule', 'carbon', 'atom'], ['leader'], ['colorful']]


In [ ]:
def preprocess_columns(text, stop_words):
    '''
    text = the tolist() from column value
    '''
    clues = remove_punc(text)
    clues = remove_stopwords(clues, stop_words)
    clues = make_bigrams(clues)
    clues = lemmatization(clues)
    clues = remove_stopwords(clues, stop_words)
    return clues

In [ ]:
clean_clues = preprocess_columns(text, stop_words)

In [ ]:
clean_clues

In [ ]:
# Create Dictionary
id2word = corpora.Dictionary(clean_clues)

# Create Corpus
texts = clean_clues

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]
#tfidf
# for doc in corpus:
#     print([[id2word[idx], freq] for idx, freq in doc])



In [ ]:
# corpus

In [ ]:
# View
print(corpus[:1])



In [ ]:
# Human readable format of corpus (term-frequency)
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

## Build the LDA model

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=13, 
                                           random_state=123,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [ ]:
# Print the keywords in the 10 topics

# pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
pprint(lda_model.print_topics())
# the weights of top 10 keywords that contribute to each
# topic reflect how important that keyword is to that topic

### Get model perplexity and coherence score

In [ ]:
#Perplexity = a measure of how good the model is. 
# The lower, the better
print ("Perplexity:", lda_model.log_perplexity(corpus))

In [ ]:
coherence_model = CoherenceModel(model=lda_model, texts=clue_lemmatized,
                     dictionary=id2word, coherence='c_v')

coherence_lda = coherence_model.get_coherence()

print("Coherence Score:", coherence_lda)
                           

## Visualize the topics and keywords

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

## Using LDA Mallet model

In [ ]:
mallet_path = "~/Downloads/mallet-2.0.8/bin/mallet"
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path,
                                             corpus=corpus, 
                                             num_topics=13,
                                             id2word=id2word)

In [ ]:
# Show the topics
pprint(ldamallet.show_topics(formatted=False)

In [ ]:
coherence_model_ldamallet = CoherenceModel(model=ldamallet,
                                          text=clue_lemmatized,
                                          dictionary=id2word,
                                          coherence='c_v')

coherence_ldamallet = coherence_model_ldamallet.get_coherence()

print("Coherence Score:", coherence_ldamallet)

## picking the best number of topics for the LDA model

In [ ]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [ ]:
# Can take a long time to run.
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=clue_lemmatized, start=2, limit=40, step=6)